# Network Analysis of Characters in a Document

In this notebook, we look at pairs of characters being mentioned in the plot for the Harry Potter novels (as 
described in Wikipedia), and do some network analysis on them.

In [ ]:
%matplotlib inline

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import requests
import re
import itertools
from bs4 import BeautifulSoup
import spacy
import math

First we need to get the content from wikipedia. Normally it's not a good idea to scrape wikipedia like this
(there are better ways), but what we are doing here is a general technique that will work on most web pages. 

In [ ]:
r = requests.get('https://en.wikipedia.org/wiki/Harry_Potter')

In [ ]:
soup = BeautifulSoup(r.text)

We know that there's a section in there somewhere for the plot... let's try to find it.

In [ ]:
soup.find_all('h2')

In [ ]:
plot_starts = soup.find('h2', text="Plot")
plot_ends = plot_starts.find_next_sibling('h2')

We need to find all the content between those two tags and turn it into plaintext sentences.

In [ ]:
plot = []
sib = plot_starts.next_element
footnote = re.compile('\[\d+\]')
while sib != plot_ends:
    sib = sib.next_element
    try:
        t = sib.text
    except AttributeError:
        continue
    t = footnote.sub(' ', t)
    t = t.strip()
    if t == '':
        continue
    if len(t.split(' ')) < 3:
        continue
    plot.append(t)
plot

Now, let's analyse each of those text chunks using the Spacy library.

In [ ]:
nlp = spacy.load("en_core_web_sm")

In [ ]:
%%time
parsed_list = list(map(nlp, plot))

Let's iterate through every text chunk, breaking it into sentences, and then within those sentences, grab all the
people that are mentioned.

An improvement would be to look at the following sentences and look for "he" or "she" or "him" or "her" and match
them up to the person mentioned in the previous sentence; this is known as _coreference resolution_.

In [ ]:
sentences = []
entity_comentions = []
for p in parsed_list:
    for s in p.sents:
        sentences.append(s)
        entities = [x.root.text for x in s.ents if x.label_ == 'PERSON']
        entity_comentions.append(entities)
entity_comentions

As a reminder, the `itertools.combinations` function can take a 3-element list and give you pairs of elements that
appear in it.

In [ ]:
list(itertools.combinations(['Ron', 'Weasley', 'Riddle'], 2))

A few words are getting mis-identified as a person. A Pensieve is a thing; Surrey is a place. We don't want
Rowling included in our graph, and so on.

In [ ]:
kill_list = ['Pensieve', 'Surrey', 'awakens', 'Muggles', 'Rowling']

This is probably the most complex cell in this notebook. We look at the people mentioned in each sentence. Then
we use `itertools.combinations` to get all the pairs of people mentioned. If they aren't in alphabetical order,
put them in order, and if anyone is on the kill list, ignore them. Finally, we'll store the pair in a dictionary and count the number of times that pair is mentioned together.

(Improvements might include mapping "Harry" to "Potter", and "Weasley" to either "Ron" or "Ginny" as appropriate.)

In [ ]:
pairs = {}
for comention in entity_comentions:
    if len(comention) < 2:
        continue
    for pair in itertools.combinations(comention, 2):
        # Get them in alphabetical order
        if pair[0] > pair[1]:
            pair = (pair[1], pair[0])
        if pair[0] == pair[1]:
            continue
        if pair[0] in kill_list:
            continue
        if pair[1] in kill_list:
            continue
        pairs[pair] = pairs.get(pair, 0) + 1
pairs

## WHAT IS A GRAPH?

A graph consists of a nodes (or vertices) and are connected by edges. Many types of real-world problems involve dependencies between observations.

- Town planners are looking at vehicular flows through a city

- Sociologist want to understand how people influence others that they know (if at all)

- Biologists want to know how proteins regulate the actions of other proteins

- Credit card fraud: vendors and card users are nodes in the network, purchases are edges

- Social media networks want to identify groups of close friends

(DiGraphs have arrows, MultiGraphs have parallel lines between nodes)

We will create a graph showing the connections between characters in Harry Potter.

The `networkx` library is a convenient library for graph operations. It works adequately for graphs with a few
thousands nodes for most algorithms.

But be aware it also implements algorithms for some complex problems that scale so poorly that all the world's supercomputers together can't solve them for a thousand node graph.

In [ ]:
import networkx as nx
graph = nx.Graph()

The `.add_edge` method also creates nodes (it calls the `.add_node` method) while it creates the edges. You can
optionally specify a weight.

Here, the weight of the edge is how often the two characters are mentioned together in a sentence.

In [ ]:
for (pair, weight) in pairs.items():
    graph.add_edge(pair[0], pair[1], weight=weight)

For small graphs, it's easy to visualise them with the built-in functions in the networkx library. For larger graphs,
try exporting to graphviz.

In [ ]:
nx.draw_networkx(graph)

The degree of a node is the number of edges that have one end in the node.

In [ ]:
nx.degree(graph, 'Harry')

In [ ]:
nx.degree(graph, 'Hagrid')

# Characterising Graphs

When we have a graph, we'd like to be able to label the graph if it has certain common characteristics.

### Is it nearly a lattice?

- A lattice has every possible edge

- It has the maximum possible _density_

Density – connectedness of the graph: ratio of edges to total possible number of edges

In [ ]:
nx.density(graph)

### Are there any islands?

An _island_ is when there is at least one pair of nodes that have no paths between them.

If there are no islands, then the `connected_components` function will return a one-element list. That one element
will be the whole graph

In [ ]:
list(nx.connected_components(graph))

If you see a document where there are islands of character co-mentions, it means that there are two completely different
plots that don't overlap. Or, you have accidentally included a Tolkien book in your analysis of Harry Potter. (An
easy mistake to make.)

Almost all network analysis assumes full connectivity, which might mean creating a subgraph to analyse:
`nx.subgraph(graph, ['Harry'])`

### World-size

#### Average clustering coefficient.

If three nodes are connected by at least two edges, what is the probability that they are connected by three?

If character A and B get mentioned together, and B and C get mentioned together, what's the probability that A and C
will be mentioned?

In [ ]:
nx.average_clustering(graph)

#### Average shortest path

Calculate the shortest path between each pair of nodes, divide by the number of pairs. (Computationally expensive)

In [ ]:
nx.average_shortest_path_length(graph)

|                                                                 | Average clustering coefficient is small | Average clustering coefficient is large |
|-----------------------------------------------------------------|-----------------------------------------|-----------------------------------------|
| Average shortest path is small (less than log(number of nodes)) | Random graph                            | Small world                             |
| Average shortest path is large                                  | Chain                                   | Chain of cliques                                 |

**Random graphs** are uninteresting: nodes are randomly connected to each other. There is probably no point in analysing
this network any further.

**Chains** are also uninteresting, although it can be interesting to look at what process is attaching nodes to either end:
is one end favoured? Why?

**Small world networks** are very interesting and very well studied. You will see these appear very regularly.

**Chains of cliques** are very, very rare and we have very few tools that can study them well.

In [ ]:
math.log(len(graph))

### Small world networks

Have hubs (high degree nodes), and many low-degree nodes. They appear to be more robust against failure of an individual node.

Examples:

- Website navigation menus

- Power grids

- Telephone call graphs

- Social networks

- Six degrees of separation (among living people)

Whenever you see a small world network, it means that there is an **effect joining clusters which is not just spatial or temporal proximity**.


#### Harry Potter is a small world network

J.K. Rowling didn't just write random characters together -- she had a plot in mind. So it's not surprising
that we see a small-world network here.

If we pick a random character, we are likely to pick a character of little importance (e.g. Hagrid could be
dropped from the novels without making much difference to the plot.) But if we take a high-degree node out
(such as Harry Potter), the novels would be very, very different!

#### Ultra-small worlds are called "Scale Free"

In an ultra-small world, the distribution of the "degree” of a node follows a _power law_. E.g. double the number of connections = half as many nodes have that number.

You can see if this is happening: do a log-log plot and see if it looks straight

Scale-free networks are generally **formed by preferential attachment** -- a new node is added to the network with a probability related to the degree of existing nodes. This happens in internet links for example -- when you add a new link, you
probably want to link to an important core network; it's rare to make a link to a minor router.

In [ ]:
degrees = pd.Series([nx.degree(graph, node) for node in graph])
degree_counts = degrees.value_counts().reset_index()
degree_counts.columns = ['degree', 'freq']
degree_counts.plot.scatter(x='degree', y='freq', logx=True, logy=True)
plt.xlim(0.5, 100)
plt.ylim(0.5, 100)

Harry Potter characters don't quite appear to be scale-free. But we don't have much data, so it's hard to say.

# Characterising Nodes

#### What are the most important nodes?

We need to calculate some measures of importance (or centrality).

There are many techniques, and they don’t generally agree with each other very much. Most only work for the identifying a few important nodes, and can’t distinguish lesser-importance nodes.

It's common to add these importance measures as features into a dataframe for supervised learning problems, as they 
express some concept of how well embedded into a product community a customer is.

The `networkx` algorithms all return a dictionary with nodes as keys, and the importance measure as the value. To make it
a little simpler to work with them, we'll create this convenience function.

In [ ]:
def nx_metric(data):
    return pd.Series(index=list(data.keys()), data=list(data.values())).sort_values(ascending=False)

### Degree Centrality

The number of edges a node has (divided by the number of other nodes)

In [ ]:
nx.degree_centrality(graph)

In [ ]:
nx_metric(nx.degree_centrality(graph)).head(5)

Degree centrality says that the Harry Potter novels are about Harry, Voldemort, Ron, Ginny and Pettigrew.

#### Closeness Centrality

The reciprocal of the sum of the shortest path distances from one node to all n-1 other nodes. Since the sum of distances depends on the number of nodes in the graph, closeness is normalized by the sum of minimum possible distances n-1. Higher values of closeness indicate higher centrality

In [ ]:
nx_metric(nx.closeness_centrality(graph)).head(5)

#### Betweenness Centrality

The sum of the fraction of all-pairs shortest paths that pass through the node

In [ ]:
nx_metric(nx.betweenness_centrality(graph)).head(5)

#### Eigenvector centrality

Computes the centrality for a node based on the centrality of its neighbours. A way of imagining this is: if we let
some ants loose on the network and they just randomly moved from one node to the next in proportion to the degree of
the nodes and the weights of the edges, where would the ants end up?

In [ ]:
nx_metric(nx.eigenvector_centrality(graph)).head(5)

#### Page Rank

Count the number and quality of links to a page to determine a rough estimate of how important the website is. The underlying assumption is that more important websites are likely to receive more links from other websites.

In [ ]:
nx_metric(nx.pagerank(graph)).head(5)

# HOW CAN WE FIND COMMUNITIES?

The criteria for finding good communities is similar to that for finding good clusters. 

We want to maximize intra-community edges while minimizing inter-community edges. 

Formally, the algorithm tries to maximize the modularity of network, or the fraction of edges that fall within the community minus the expected fraction of edges if the edges were distributed by random. Good communities should have a high number of intra-community edges, so by maximizing the modularity, we detect dense communities that have a high fraction of intra-community edges.

In [ ]:
for c in nx.clique.find_cliques(graph):
    print(c)

In [ ]:
nx.clique.cliques_containing_node(graph,'Hagrid')

This does find the trio correctly...

In [ ]:
nx.clique.cliques_containing_node(graph,'Granger')

#### Similarity

Or, we could ask for groups of nodes with a high Jaccard coefficient. The Jaccard coefficient for two nodes is
the ratio of (neighbours in common) to (neighbours of either).

In [ ]:
list(nx.jaccard_coefficient(graph, 
                               [('Harry', "Ron")]))

In [ ]:
list(nx.jaccard_coefficient(graph, 
                               [('Voldemort', "Hagrid")]))

So Harry and Ron and more similar to each other than Voldemort and Hagrid are.

# Everything else

In [ ]:
#nx.write_edgelist(graph, 'edgelist.txt')
#graph = nx.read_edgelist('edgelist.txt')

In [ ]:
nx.__version__

In [ ]:
nx.to_pandas_edgelist(graph)

In [ ]:
nx.to_pandas_adjacency(graph)

In [ ]:
#networkx.from_pandas_edgelist()
#networkx.from_pandas_adjacency()

# Lots more algorithms to explore

Start here: http://networkx.readthedocs.io/en/stable/reference/algorithms.html and have some fun!